In [6]:
%reload_ext autoreload
%autoreload 2

import train_sort_atl as ts
from train_sort_atl import AverageMeter, mainArguments, masked_accuracy

import torch
import numpy as np
import matplotlib.pyplot as plt

In [7]:
import argparse
import random
import warnings

import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import Adam
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

import dataset
import model

In [44]:
userArgs={
    'low':0,
    'high':100,
    'min_length':5,
    'max_length':10,
    'train_samples':100000,
    'test_samples':1000,
    'emb_dim':8,
    'batch_size':256,
    'epochs':100,
    'lr':5e-3,
    'wd':1e-5,
    'workers':4,
    'no_cuda':False,
    'seed':None
}
args = mainArguments(**userArgs)

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
cudnn.benchmark = True if use_cuda else False

train_set = dataset.IntegerSortDataset(num_samples=args.train_samples, high=args.high, min_len=args.min_length, max_len=args.max_length, seed=1)
train_loader = DataLoader(dataset=train_set, batch_size=args.batch_size, shuffle=True, num_workers=args.workers, collate_fn=dataset.sparse_seq_collate_fn)

test_set = dataset.IntegerSortDataset(num_samples=args.test_samples, high=args.high, min_len=args.min_length, max_len=args.max_length, seed=2)
test_loader = DataLoader(dataset=test_set, batch_size=args.batch_size, shuffle=False, num_workers=args.workers, collate_fn=dataset.sparse_seq_collate_fn)

net = model.PointerNet(input_dim=args.high, embedding_dim=args.emb_dim, hidden_size=args.emb_dim).to(device)
optimizer = Adam(net.parameters(), lr=args.lr, weight_decay=args.wd)

train_loss = AverageMeter()
train_accuracy = AverageMeter()
test_loss = AverageMeter()
test_accuracy = AverageMeter()

for epoch in range(args.epochs):
    # Train
    net.train()
    for batch_idx, (seq, length, target) in enumerate(train_loader):
        seq, length, target = seq.to(device), length.to(device), target.to(device)
        
        optimizer.zero_grad()
        log_pointer_score, argmax_pointer, mask = net(seq, length)
        
        raise ValueError('hi')
        
        unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
        loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
        assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

        loss.backward()
        optimizer.step()

        train_loss.update(loss.item(), seq.size(0))

        mask = mask[:, 0, :]
        train_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())

        # if batch_idx % 20 == 0:
        #     print('Epoch {}: Train [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'
        #           .format(epoch, batch_idx * len(seq), len(train_loader.dataset),
        #                   100. * batch_idx / len(train_loader), train_loss.avg, train_accuracy.avg))

    # Test
    net.eval()
    for seq, length, target in test_loader:
        seq, length, target = seq.to(device), length.to(device), target.to(device)

        log_pointer_score, argmax_pointer, mask = net(seq, length)
        unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
        loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
        assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

        test_loss.update(loss.item(), seq.size(0))

        mask = mask[:, 0, :]
        test_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())
    print('Epoch {}: Test\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(epoch, test_loss.avg, test_accuracy.avg))


Input_seq.shape: torch.Size([256, 10, 100])
Embedded.shape: torch.Size([256, 10, 8])
encoder_outputs.shape: torch.Size([256, 10, 16])
encoder_hidden.shapes: torch.Size([2, 256, 8]) -- torch.Size([2, 256, 8])
post bidi - encoder_outputs.shape: torch.Size([256, 10, 8])
encoder_h_n.shape: torch.Size([1, 2, 256, 8])
encoder_c_n.shape: torch.Size([1, 2, 256, 8])
decoder_input.shape: torch.Size([256, 8])
decoder_hidden.shape: torch.Size([256, 8]) -- torch.Size([256, 8])
input_lengths.shape: torch.Size([256])
each_len_tensor.shape: torch.Size([256, 10, 10])


ValueError: hi

In [5]:
# test_set = IntegerSortDataset(num_samples=args.test_samples, high=args.high, min_len=args.min_length, max_len=args.max_length, seed=2)
# test_loader = DataLoader(dataset=test_set, batch_size=args.batch_size, shuffle=False, num_workers=args.workers, collate_fn=sparse_seq_collate_fn)

# model = PointerNet(input_dim=args.high, embedding_dim=args.emb_dim, hidden_size=args.emb_dim).to(device)
# optimizer = Adam(model.parameters(), lr=args.lr, weight_decay=args.wd)

# train_loss = AverageMeter()
# train_accuracy = AverageMeter()
# test_loss = AverageMeter()
# test_accuracy = AverageMeter()

# for epoch in range(args.epochs):
#     # Train
#     model.train()
#     for batch_idx, (seq, length, target) in enumerate(train_loader):
#         seq, length, target = seq.to(device), length.to(device), target.to(device)

#         optimizer.zero_grad()
#         log_pointer_score, argmax_pointer, mask = model(seq, length)

#         unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
#         loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
#         assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

#         loss.backward()
#         optimizer.step()

#         train_loss.update(loss.item(), seq.size(0))

#         mask = mask[:, 0, :]
#         train_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())

#         # if batch_idx % 20 == 0:
#         #     print('Epoch {}: Train [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}'
#         #           .format(epoch, batch_idx * len(seq), len(train_loader.dataset),
#         #                   100. * batch_idx / len(train_loader), train_loss.avg, train_accuracy.avg))

#     # Test
#     model.eval()
#     for seq, length, target in test_loader:
#         seq, length, target = seq.to(device), length.to(device), target.to(device)

#         log_pointer_score, argmax_pointer, mask = model(seq, length)
#         unrolled = log_pointer_score.view(-1, log_pointer_score.size(-1))
#         loss = F.nll_loss(unrolled, target.view(-1), ignore_index=-1)
#         assert not np.isnan(loss.item()), 'Model diverged with loss = NaN'

#         test_loss.update(loss.item(), seq.size(0))

#         mask = mask[:, 0, :]
#         test_accuracy.update(masked_accuracy(argmax_pointer, target, mask).item(), mask.int().sum().item())
#     print('Epoch {}: Test\tLoss: {:.6f}\tAccuracy: {:.6f}'.format(epoch, test_loss.avg, test_accuracy.avg))
